# Usage example on notMNIST

### First download the dataset from http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat

- [ ] MLD issue "post AKNN" contains more margin experiments to run

In [1]:
import numpy as np, scipy as sp, time, scipy.io, sklearn
import aknn_alg

notMNIST_small = scipy.io.loadmat("notMNIST_small.mat")['images'].reshape(784, 18724)
nmn = (notMNIST_small.T - 255.0/2)/255.0
labels = scipy.io.loadmat("notMNIST_small.mat")['labels'].astype(int)
labels_to_symbols = { 0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J' }
labels = np.array([labels_to_symbols[x] for x in labels])

In [2]:
import importlib
importlib.reload(aknn_alg)

# Calculate list of exact Euclidean nearest neighbors for each point - use more neighbors for less abstaining at a given parameter setting.
itime = time.time()
nbrs_list = aknn_alg.calc_nbrs_exact(nmn, k=1000, use_nndescent=False)
print('Neighbor indices computed. Time:\t {}'.format(time.time() - itime))

Neighbor indices computed. Time:	 8.706866025924683


In [ ]:
"""
# Using the Annoy package

%%time

#a = calc_nbrs_pynndescent(raw_data)
from annoy import AnnoyIndex

import random

dim = nmn.shape[1]
t = AnnoyIndex(dim, 'euclidean')  # Length of item vector that will be indexed

for i in range(nmn.shape[0]):
    t.add_item(i, nmn[i, :])

t.build(10) # 10 trees
t.save('test.ann')

u = AnnoyIndex(dim, 'euclidean')
u.load('test.ann') # super fast, will just mmap the file

results = []
for i in range(nmn.shape[0]):
    results.append(u.get_nns_by_item(i, 1000)) # will find the 1000 nearest neighbors

np.array(results)[:, 1:]    # compare with nbrs_list
"""

CPU times: user 1.77 s, sys: 453 ms, total: 2.23 s
Wall time: 1.66 s


True

## Make AKNN predictions

In [3]:
itime = time.time()
aknn_predictions = aknn_alg.predict_nn_rule(nbrs_list, labels)
print('AKNN predictions made. Time:\t {}'.format(time.time() - itime))

AKNN predictions made. Time:	 3.185471296310425


In [5]:
import importlib
importlib.reload(aknn_alg)

ref_data = nmn
margin = 1.0
query_data = None
max_k = 1000

# itime = time.time()
if query_data is None:
    query_data = ref_data
nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=max_k).fit(ref_data)
distances, indices = nbrs.kneighbors(query_data)
distinct_labels = np.unique(labels)
rngarr = np.arange(indices.shape[1])+1
query_nbrs = labels[indices]
fracs_labels = [np.cumsum(query_nbrs == i, axis=1)/rngarr for i in distinct_labels]
# print("Clustering computed. Time: {}".format(time.time() - itime))
thresholds = margin/np.sqrt(np.arange(indices.shape[1]) + 1)
numlabels_predicted = np.add.reduce([f > (thresholds + 1.0/len(distinct_labels)) for f in fracs_labels])
adaptive_k = np.argmax(numlabels_predicted > 0, axis=1)

# pred_labels = np.zeros(fracs_labels[0].shape[0]).astype(str)
# for i in range(fracs_labels[0].shape[0]):
#     if adaptive_k[i] == 0:
#         pred_labels[i] = '?'
#     else:
#         lst = [f[i, adaptive_k[i]] for f in fracs_labels]
#         pred_labels[i] = distinct_labels[np.argmax(lst)]

In [38]:
itime = time.time()
aknn_predictions_new = aknn_alg.aknn_predict(nmn, labels, max_k=1000)
print('AKNN predictions made. Time:\t {}'.format(time.time() - itime))

AKNN predictions made. Time:	 12.703313827514648


In [43]:
np.mean(aknn_predictions[0] == aknn_predictions_new[0])

0.9654988250373852

## Comparison with k-NN

In [7]:
kvals = [3,5,7,8,10,30,100]
for i in range(len(kvals)):
    knn_predictions = aknn_alg.knn_rule(nbrs_list, labels, k=kvals[i])
    aknn_cov_ndces = aknn_predictions[1] <= kvals[i]
    aknn_cov = np.mean(aknn_cov_ndces)
    aknn_condacc = np.mean((aknn_predictions[0] == labels)[aknn_cov_ndces])
    print('{}-NN accuracy: \t\t{}'.format(kvals[i], np.mean(knn_predictions == labels)))
    print('AKNN accuracy (k <= {}): \t{} \t\t Coverage: \t{}\n'.format(
        kvals[i], aknn_condacc, aknn_cov))
print('Overall AKNN accuracy: {}'.format(np.mean(aknn_predictions[0] == labels)))

3-NN accuracy: 		0.8750267036957915
AKNN accuracy (k <= 3): 	0.9701739850869926 		 Coverage: 	0.838015381328776

5-NN accuracy: 		0.8833048493911557
AKNN accuracy (k <= 5): 	0.9450811565536099 		 Coverage: 	0.9180196539201025

7-NN accuracy: 		0.8836787011322367
AKNN accuracy (k <= 7): 	0.9408167974157822 		 Coverage: 	0.9258705404828028

8-NN accuracy: 		0.8834650715659047
AKNN accuracy (k <= 8): 	0.9362406530053086 		 Coverage: 	0.935644093142491

10-NN accuracy: 		0.8822901089510788
AKNN accuracy (k <= 10): 	0.9322341209133662 		 Coverage: 	0.9425870540482802

30-NN accuracy: 		0.8767891476180303
AKNN accuracy (k <= 30): 	0.9158672400485169 		 Coverage: 	0.9687032685323649

100-NN accuracy: 		0.858577227088229
AKNN accuracy (k <= 100): 	0.9071918180829072 		 Coverage: 	0.9817346720786156

Overall AKNN accuracy: 0.8925977355265969
